In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from sentinelhub import SHConfig
from sentinelhub import (
    DataCollection,
    MimeType,
    SentinelHubRequest,
    bbox_to_dimensions,
)
import folium
from tqdm import tqdm
import math

In [2]:
from sentinelhub import SHConfig
from dotenv import load_dotenv
import os
import numpy as np
import datetime
import os
import matplotlib.pyplot as plt
from sentinelhub import CRS, BBox, bbox_to_dimensions
import cv2
import ee

config_sentinel = SHConfig(sh_client_id=os.environ.get("sh_client_id"), sh_client_secret=os.environ.get("sh_client_secret"))

In [49]:
evalscript_lsm = """
function setup() {
  return {
    input: ["B02", "B03", "B04", "B08", "B11"],
    output: { bands: 3 },
  };
}

function calcIndex(x, y) {
  return (x - y) / (x + y);
}

function clip(a) {
  return Math.max(0, Math.min(1, a));
}

function evaluatePixel(sample) {
  let bRatio = (sample.B03 - 0.175) / (0.39 - 0.175);
  let NDGR = calcIndex(sample.B03, sample.B04);
  let NDVI = calcIndex(sample.B08, sample.B04);
  let NDWI = calcIndex(sample.B03, sample.B08);
  let gain = 2.5;

  // Handle invalid or non-land areas
  if (!isFinite(NDGR) || !isFinite(NDVI) || !isFinite(NDWI) || sample.B02 === 0) {
    return [0, 0, 0]; // Black
  }

  // Cloud Detection - Turn Clouds to Red
  if (sample.B11 > 0.1 && bRatio > 0.001) {
    return [0, 0, 0]; // Pure Red
  }
  if (sample.B11 > 0.1 && bRatio > 0 && NDGR > 0) {
    return [1, 0, 0]; // Pure Red
  }

  // Water Detection
  if (NDWI > 0.15) {
    return [0, 0.2, 1.0 * NDWI];
  }

  // Other regions
  if (sample.B11 > 0.95 || NDVI < 0.1) {
    return [1.5, 0.7, -1]; // Special visualization
  }

  // Normal RGB Processing
  return [sample.B04, sample.B03, sample.B02].map(a => clip(gain * a));
}
"""

In [50]:
def get_suseptibility_mapping(cordinates, box_dim=400, date_start = "2024-04-12", date_end = "2024-04-12", res=2100):
    min_lat, min_lon, max_lat, max_lon  = cordinates

    cords = [min_lon, min_lat, max_lon, max_lat]

    bbox = BBox(bbox=cords, crs=CRS.WGS84)
    size = bbox_to_dimensions(bbox, resolution=box_dim*1000/res)

    request_lms_color = SentinelHubRequest(
            evalscript=evalscript_lsm,
            input_data=[
                SentinelHubRequest.input_data(
                    data_collection=DataCollection.SENTINEL2_L2A,
                    time_interval=(date_start, date_end),
                )
            ],
            responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
            bbox=bbox,
            size=size,
            config=config_sentinel,
        )

    lms_response = request_lms_color.get_data()
    return lms_response[0]

In [51]:
import math

def generate_grid(top_left_lat, top_left_lon, grid_side = 9, distance=400000):
    R = 6371000  # Earth's radius in meters
    grid = []
    
    # Convert top-left latitude to radians
    top_left_lat_rad = math.radians(top_left_lat)

    # Compute shifts in degrees
    delta_lat = (distance / R) * (180 / math.pi)
    delta_lon = (distance / (R * math.cos(top_left_lat_rad))) * (180 / math.pi)

    # Generate grid (9x9)
    for row in range(grid_side):  # Move downward
        for col in range(grid_side):  # Move right
            min_lat = top_left_lat - (row * delta_lat)  # Move south
            min_lon = top_left_lon + (col * delta_lon)  # Move east
            max_lat = min_lat + delta_lat
            max_lon = min_lon + delta_lon
            grid.append([min_lat, min_lon, max_lat, max_lon])
    
    return grid

# Test the function
lat, lon = 11.464868870490513, 76.13516957235045
distance = 2 * 1000  # 10 km
g = generate_grid(lat, lon, distance=distance, grid_side=9)

In [52]:
grid = 1
box_dim = 5 # km
min_lat, min_lon  = 33.23119, 75.18916

g = generate_grid(min_lat, min_lon, distance=box_dim*1000, grid_side=grid)

m = folium.Map(
    location=(min_lat, min_lon),
    zoom_start=15,
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri'
)
for i in range(len(g)):
    folium.Rectangle([(g[i][:2]), (g[i][2:])], color='red', fill='pink',fill_opcity=0.5).add_to(m)

m

In [54]:
_box_dim = 1000 if box_dim < 100 else box_dim

canvas = np.zeros(((grid) * _box_dim, (grid) * _box_dim, 3), dtype=np.uint8)

row = 0
col = 0

for idx, i in enumerate(range(len(g))):

    y_start = _box_dim * col
    y_end = _box_dim * (col + 1)
    x_start = _box_dim * row
    x_end = _box_dim * (row + 1)

    print(col, row, g[idx], "->", y_start, y_end, x_start, x_end) 

    canvas[y_start:y_end, x_start:x_end] = cv2.resize(get_suseptibility_mapping(g[idx], date_start="2024-05-06", date_end="2024-05-06", res=1000, box_dim=box_dim), (_box_dim,_box_dim))
    plt.imsave("mapping5.png", canvas)
    row += 1  # Move to the next column

    if (idx + 1) % math.sqrt(len(g)) == 0:
        print("----") 
        col += 1  # Move to the next row
        row = 0  # Reset column position

0 0 [33.23119, 75.18916, 33.276156080295934, 75.24291720494928] -> 0 1000 0 1000
----


In [72]:
math.sqrt(len(g))

9.0

In [71]:
y_start, y_end, x_start, x_end

(0, 1000, 3000, 4000)

In [73]:
len(g)

81